In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Load training datasets and testing datasets 
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv', low_memory = False)
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv', low_memory = False)
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv', low_memory = False)

## Evaluate 'label' prediction models: 
Includes all those models that have been trained to identify between normal traffic and attack traffic. 

In [3]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Transform to One Hot Encoding the Categories - normal, attack
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'attack', data['attack_cat'].replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 1).replace('normal', 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
def evaluate_detection(model, x, y): 
    # Need to return FNR, F1, Precision, Recall, Accuracy, number of false positives, number of false negatives, number of true positives, number of true negatives
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y, axis=1)
    report = classification_report(y_true, y_pred, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    accuracy = report['accuracy']
    FNR = fn / (fn + tp)

    
    return FNR, f1, precision, recall, accuracy, cm

In [5]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcom, ycom = preprocessing(test_complete)

#### 910, 911, 912, 913, 914, 915, 916, 917

In [34]:
mod910 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID910.hdf5')
mod911 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID911.hdf5')
mod912 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID912.hdf5')
mod913 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID913.hdf5')
mod914 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID914.hdf5')
mod915 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID915.hdf5')
mod916 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID916.hdf5')
mod917 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID917.hdf5')

In [28]:
evaluate_detection(mod910, xbasic, ybasic)

(0.8415586831173663,
 0.5063974935793695,
 0.7144701972247314,
 0.593298080574951,
 0.593298080574951,
 array([[71010,  1390],
        [54294, 10222]], dtype=int64))

In [36]:
evaluate_detection(mod910, xplus, yplus)

(0.6102725280178652,
 0.6953794476261614,
 0.8076431743442479,
 0.7279117924871031,
 0.7279117924871031,
 array([[30649,   222],
        [14757,  9424]], dtype=int64))

In [29]:
evaluate_detection(mod911, xbasic, ybasic)

(0.2876960753921508,
 0.8560237382588983,
 0.8846263110674074,
 0.8596073504922727,
 0.8596073504922727,
 array([[71739,   661],
        [18561, 45955]], dtype=int64))

In [37]:
evaluate_detection(mod911, xplus, yplus)

(0.7614242587155204,
 0.5989273184549058,
 0.7781947755840517,
 0.6627007193199157,
 0.6627007193199157,
 array([[30714,   157],
        [18412,  5769]], dtype=int64))

In [30]:
evaluate_detection(mod912, xbasic, ybasic)

(0.8828507657015314,
 0.4760500893877768,
 0.7379895399508547,
 0.5804653948406322,
 0.5804653948406322,
 array([[71917,   483],
        [56958,  7558]], dtype=int64))

In [38]:
evaluate_detection(mod912, xplus, yplus)

(0.9368926016293785,
 0.46206753347746077,
 0.7540550333979036,
 0.5879350432318535,
 0.5879350432318535,
 array([[30841,    30],
        [22655,  1526]], dtype=int64))

In [31]:
evaluate_detection(mod913, xbasic, ybasic)

(0.8535402070804141,
 0.49973910462752014,
 0.7355091256181312,
 0.5923193783049461,
 0.5923193783049461,
 array([[71649,   751],
        [55067,  9449]], dtype=int64))

In [39]:
evaluate_detection(mod913, xplus, yplus)

(0.8952483354699972,
 0.49776575214342295,
 0.7580904276221205,
 0.6056455714597109,
 0.6056455714597109,
 array([[30809,    62],
        [21648,  2533]], dtype=int64))

In [32]:
evaluate_detection(mod914, xbasic, ybasic)

(0.8739382478764958,
 0.4837946867218398,
 0.7414682594253333,
 0.5847015688451314,
 0.5847015688451314,
 array([[71922,   478],
        [56383,  8133]], dtype=int64))

In [40]:
evaluate_detection(mod914, xplus, yplus)

(0.92225300856044,
 0.47302792034124125,
 0.7193566642547569,
 0.5910593620576909,
 0.5910593620576909,
 array([[30659,   212],
        [22301,  1880]], dtype=int64))

In [33]:
evaluate_detection(mod915, xbasic, ybasic)

(0.8582522165044331,
 0.49617727724775007,
 0.7372821739744763,
 0.5906614274445645,
 0.5906614274445645,
 array([[71726,   674],
        [55371,  9145]], dtype=int64))

In [41]:
evaluate_detection(mod915, xplus, yplus)

(0.7052231090525619,
 0.6368594826811318,
 0.7895831169954107,
 0.6871684952408632,
 0.6871684952408632,
 array([[30702,   169],
        [17053,  7128]], dtype=int64))

In [34]:
evaluate_detection(mod916, xbasic, ybasic)

(0.8445811891623783,
 0.5035784985272762,
 0.7104288177094473,
 0.5913698910280756,
 0.5913698910280756,
 array([[70941,  1459],
        [54489, 10027]], dtype=int64))

In [42]:
evaluate_detection(mod916, xplus, yplus)

(0.6063024688805261,
 0.6969376973614473,
 0.8061525739118192,
 0.7287473661265712,
 0.7287473661265712,
 array([[30599,   272],
        [14661,  9520]], dtype=int64))

In [35]:
evaluate_detection(mod917, xbasic, ybasic)

(0.07235414470828942,
 0.9641574953615748,
 0.9661124994480643,
 0.9642627596482515,
 0.9642627596482515,
 array([[72175,   225],
        [ 4668, 59848]], dtype=int64))

In [43]:
evaluate_detection(mod917, xplus, yplus)

(0.7047682064430751,
 0.6362723102708954,
 0.7859974529417739,
 0.6862965923127226,
 0.6862965923127226,
 array([[30643,   228],
        [17042,  7139]], dtype=int64))

In [35]:
evaluate_detection(mod910, xcom, ycom)

(0.7881117125160426,
 0.5280034936318392,
 0.7452920639657324,
 0.598581283262072,
 0.598581283262072,
 array([[70629,  1056],
        [56495, 15189]], dtype=int64))

In [36]:
evaluate_detection(mod911, xcom, ycom)

(0.3008342168405781,
 0.8414632128852411,
 0.8768048304643389,
 0.8448269849130565,
 0.8448269849130565,
 array([[71003,   682],
        [21565, 50119]], dtype=int64))

In [37]:
evaluate_detection(mod912, xcom, ycom)

(0.846757993415546,
 0.4814320830692171,
 0.7450890282096714,
 0.5726482014940468,
 0.5726482014940468,
 array([[71115,   570],
        [60699, 10985]], dtype=int64))

In [38]:
evaluate_detection(mod913, xcom, ycom)

(0.8055354053903242,
 0.5155912259094523,
 0.7512245540767714,
 0.5922340254866812,
 0.5922340254866812,
 array([[70968,   717],
        [57744, 13940]], dtype=int64))

In [39]:
evaluate_detection(mod914, xcom, ycom)

(0.8376625188326544,
 0.4895201075654078,
 0.7495894896071347,
 0.5775097824494835,
 0.5775097824494835,
 array([[71160,   525],
        [60047, 11637]], dtype=int64))

In [40]:
evaluate_detection(mod915, xcom, ycom)

(0.8193878689805256,
 0.5040938597050593,
 0.7473728475059096,
 0.5853148170106508,
 0.5853148170106508,
 array([[70969,   716],
        [58737, 12947]], dtype=int64))

In [41]:
evaluate_detection(mod916, xcom, ycom)

(0.7920316946598962,
 0.5253266440145772,
 0.7469744958772219,
 0.5972630066471831,
 0.5972630066471831,
 array([[70721,   964],
        [56776, 14908]], dtype=int64))

In [42]:
evaluate_detection(mod917, xcom, ycom)

(0.13743652697952125,
 0.929220949557373,
 0.9373860293404368,
 0.929538463684618,
 0.929538463684618,
 array([[71435,   250],
        [ 9852, 61832]], dtype=int64))

#### 900

In [43]:
mod_900 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID900.hdf5')
mod_900_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id900.hdf5')
mod_900_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id900.hdf5')
mod_900_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id900.hdf5')


In [45]:
evaluate_detection(mod_900, xbasic, ybasic)

(0.5132990265980532,
 0.7356653483572342,
 0.8262189548215215,
 0.7546378801600981,
 0.7546378801600981,
 array([[71922,   478],
        [33116, 31400]], dtype=int64))

In [49]:
evaluate_detection(mod_900, xplus, yplus)

(0.8948347876431909,
 0.49782254466752396,
 0.7540072721991109,
 0.6053731017946669,
 0.6053731017946669,
 array([[30784,    87],
        [21638,  2543]], dtype=int64))

In [46]:
evaluate_detection(mod_900_0, xbasic, ybasic)

(0.4534534069068138,
 0.7693972311368287,
 0.8402191759738504,
 0.782545502351807,
 0.782545502351807,
 array([[71882,   518],
        [29255, 35261]], dtype=int64))

In [50]:
evaluate_detection(mod_900_0, xplus, yplus)

(0.8790372606591952,
 0.5106748244833054,
 0.7555097420945571,
 0.6119668676887307,
 0.6119668676887307,
 array([[30765,   106],
        [21256,  2925]], dtype=int64))

In [47]:
evaluate_detection(mod_900_1, xbasic, ybasic)

(0.3904612809225618,
 0.8036365812035009,
 0.8565316536045363,
 0.8121622016418826,
 0.8121622016418826,
 array([[71873,   527],
        [25191, 39325]], dtype=int64))

In [51]:
evaluate_detection(mod_900_1, xplus, yplus)

(0.8893759563293495,
 0.5022470844694719,
 0.7537969265771943,
 0.6075710237593548,
 0.6075710237593548,
 array([[30773,    98],
        [21506,  2675]], dtype=int64))

In [48]:
evaluate_detection(mod_900_2, xbasic, ybasic)

(0.5140430280860562,
 0.7352193882095733,
 0.8260105773066937,
 0.7542726927459172,
 0.7542726927459172,
 array([[71920,   480],
        [33164, 31352]], dtype=int64))

In [52]:
evaluate_detection(mod_900_2, xplus, yplus)

(0.9045118067904553,
 0.4898873040465314,
 0.7547763420751633,
 0.6014858679067064,
 0.6014858679067064,
 array([[30804,    67],
        [21872,  2309]], dtype=int64))

#### 901

In [44]:
mod_901 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID901.hdf5')
mod_901_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id901.hdf5')
mod_901_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id901.hdf5')
mod_901_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id901.hdf5')


In [54]:
evaluate_detection(mod_901, xbasic, ybasic)

(4.6500093000186e-05,
 0.6711269993979918,
 0.8149807420106494,
 0.6954994303076338,
 0.6954994303076338,
 array([[30712, 41688],
        [    3, 64513]], dtype=int64))

In [55]:
evaluate_detection(mod_901, xplus, yplus)

(0.0,
 0.6762043226949995,
 0.819670241181861,
 0.6940710600886435,
 0.6940710600886435,
 array([[14029, 16842],
        [    0, 24181]], dtype=int64))

In [56]:
evaluate_detection(mod_901_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.36580572534487127,
 0.2796203190506846,
 0.5287913757340267,
 0.5287913757340267,
 array([[72400,     0],
        [64516,     0]], dtype=int64))

In [57]:
evaluate_detection(mod_901_0, xplus, yplus)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.4029478640083462,
 0.31445260225958305,
 0.5607607353048027,
 0.5607607353048027,
 array([[30871,     0],
        [24181,     0]], dtype=int64))

In [58]:
evaluate_detection(mod_901_1, xbasic, ybasic)

(0.00015500031000062,
 0.8674115613403924,
 0.8972396646715455,
 0.8686859096088112,
 0.8686859096088112,
 array([[54431, 17969],
        [   10, 64506]], dtype=int64))

In [59]:
evaluate_detection(mod_901_1, xplus, yplus)

(0.026260287002191803,
 0.871134353762791,
 0.8910863385115998,
 0.8709220373465087,
 0.8709220373465087,
 array([[24400,  6471],
        [  635, 23546]], dtype=int64))

In [60]:
evaluate_detection(mod_901_2, xbasic, ybasic)

(0.0,
 0.3855818607562937,
 0.7601955775025923,
 0.5116859972537906,
 0.5116859972537906,
 array([[ 5542, 66858],
        [    0, 64516]], dtype=int64))

In [61]:
evaluate_detection(mod_901_2, xplus, yplus)

(0.0,
 0.29160603781147343,
 0.7558249163809195,
 0.45017438058562814,
 0.45017438058562814,
 array([[  602, 30269],
        [    0, 24181]], dtype=int64))

#### 902

In [45]:
mod_902 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID902.hdf5')
mod_902_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id902.hdf5')
mod_902_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id902.hdf5')
mod_902_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id902.hdf5')


In [63]:
evaluate_detection(mod_902, xbasic, ybasic)

(0.8228811457622915,
 0.5183148087073767,
 0.7028923191363293,
 0.5977387595313916,
 0.5977387595313916,
 array([[70413,  1987],
        [53089, 11427]], dtype=int64))

In [64]:
evaluate_detection(mod_902, xplus, yplus)

(0.3011455274802531,
 0.8422116840360242,
 0.8610411045317384,
 0.8463997674925525,
 0.8463997674925525,
 array([[29697,  1174],
        [ 7282, 16899]], dtype=int64))

In [65]:
evaluate_detection(mod_902_0, xbasic, ybasic)

(0.8224316448632897,
 0.5173775718898278,
 0.6954131327694697,
 0.5961465424055625,
 0.5961465424055625,
 array([[70166,  2234],
        [53060, 11456]], dtype=int64))

In [66]:
evaluate_detection(mod_902_0, xplus, yplus)

(0.29432198833795126,
 0.8522380003359403,
 0.8732756899223758,
 0.8563721572331614,
 0.8563721572331614,
 array([[30081,   790],
        [ 7117, 17064]], dtype=int64))

In [67]:
evaluate_detection(mod_902_1, xbasic, ybasic)

(0.8225711451422902,
 0.5154894600895281,
 0.6851499246811477,
 0.5935975342545794,
 0.5935975342545794,
 array([[69826,  2574],
        [53069, 11447]], dtype=int64))

In [68]:
evaluate_detection(mod_902_1, xplus, yplus)

(0.29498366486084115,
 0.8400368672693922,
 0.8557967620070023,
 0.8438385526411393,
 0.8438385526411393,
 array([[29407,  1464],
        [ 7133, 17048]], dtype=int64))

In [69]:
evaluate_detection(mod_902_2, xbasic, ybasic)

(0.8235631471262943,
 0.5142183031479978,
 0.6818510737632014,
 0.5924435420257677,
 0.5924435420257677,
 array([[69732,  2668],
        [53133, 11383]], dtype=int64))

In [70]:
evaluate_detection(mod_902_2, xplus, yplus)

(0.30370952400645135,
 0.8331675295875275,
 0.8489368297310398,
 0.8371902928140666,
 0.8371902928140666,
 array([[29252,  1619],
        [ 7344, 16837]], dtype=int64))

#### 903

In [46]:
mod_903 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID903.hdf5')
mod_903_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id903.hdf5')
mod_903_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id903.hdf5')
mod_903_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id903.hdf5')


In [72]:
evaluate_detection(mod_903, xbasic, ybasic)

(7.750015500031e-05,
 0.30200498464953024,
 0.5855903787793216,
 0.471252446755675,
 0.471252446755675,
 array([[   11, 72389],
        [    5, 64511]], dtype=int64))

In [73]:
evaluate_detection(mod_903, xplus, yplus)

(0.0,
 0.26826037385846335,
 0.7537058860749408,
 0.4393119232725423,
 0.4393119232725423,
 array([[    4, 30867],
        [    0, 24181]], dtype=int64))

In [74]:
evaluate_detection(mod_903_0, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.36580572534487127,
 0.2796203190506846,
 0.5287913757340267,
 0.5287913757340267,
 array([[72400,     0],
        [64516,     0]], dtype=int64))

In [75]:
evaluate_detection(mod_903_0, xplus, yplus)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.4029478640083462,
 0.31445260225958305,
 0.5607607353048027,
 0.5607607353048027,
 array([[30871,     0],
        [24181,     0]], dtype=int64))

In [76]:
evaluate_detection(mod_903_1, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.30184375474744357,
 0.2220375675826312,
 0.4712086242659733,
 0.4712086242659733,
 array([[    0, 72400],
        [    0, 64516]], dtype=int64))

In [77]:
evaluate_detection(mod_903_1, xplus, yplus)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.2681015400046588,
 0.19293113164997755,
 0.43923926469519725,
 0.43923926469519725,
 array([[    0, 30871],
        [    0, 24181]], dtype=int64))

In [78]:
evaluate_detection(mod_903_2, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.30184375474744357,
 0.2220375675826312,
 0.4712086242659733,
 0.4712086242659733,
 array([[    0, 72400],
        [    0, 64516]], dtype=int64))

In [79]:
evaluate_detection(mod_903_2, xplus, yplus)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.2681015400046588,
 0.19293113164997755,
 0.43923926469519725,
 0.43923926469519725,
 array([[    0, 30871],
        [    0, 24181]], dtype=int64))

#### 904

In [47]:
mod_904 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID904.hdf5')
mod_904_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id904.hdf5')
mod_904_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id904.hdf5')
mod_904_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id904.hdf5')


In [81]:
evaluate_detection(mod_904, xbasic, ybasic)

(0.478749457498915,
 0.7562099328558912,
 0.8361581849813545,
 0.7716994361506325,
 0.7716994361506325,
 array([[72029,   371],
        [30887, 33629]], dtype=int64))

In [82]:
evaluate_detection(mod_904, xplus, yplus)

(0.9310615772714115,
 0.46691651454452276,
 0.7474853648839689,
 0.5899149894645063,
 0.5899149894645063,
 array([[30809,    62],
        [22514,  1667]], dtype=int64))

In [83]:
evaluate_detection(mod_904_0, xbasic, ybasic)

(0.46332692665385333,
 0.7645812353213383,
 0.8392353326613166,
 0.7786306932717871,
 0.7786306932717871,
 array([[71983,   417],
        [29892, 34624]], dtype=int64))

In [84]:
evaluate_detection(mod_904_0, xplus, yplus)

(0.892022662420909,
 0.5002195101009569,
 0.7554499865696139,
 0.6066809561868779,
 0.6066809561868779,
 array([[30788,    83],
        [21570,  2611]], dtype=int64))

In [85]:
evaluate_detection(mod_904_1, xbasic, ybasic)

(0.4568634137268274,
 0.7681103583724327,
 0.8406612309241416,
 0.7815887113266529,
 0.7815887113266529,
 array([[71971,   429],
        [29475, 35041]], dtype=int64))

In [86]:
evaluate_detection(mod_904_1, xplus, yplus)

(0.9131136015880237,
 0.4824062182814524,
 0.7501007097178776,
 0.5975078107970646,
 0.5975078107970646,
 array([[30793,    78],
        [22080,  2101]], dtype=int64))

In [87]:
evaluate_detection(mod_904_2, xbasic, ybasic)

(0.4655744311488623,
 0.7631092511680524,
 0.8382204366936806,
 0.7773452335738701,
 0.7773452335738701,
 array([[71952,   448],
        [30037, 34479]], dtype=int64))

In [88]:
evaluate_detection(mod_904_2, xplus, yplus)

(0.8880526032835697,
 0.5036275225527176,
 0.7579402005084992,
 0.6085882438421856,
 0.6085882438421856,
 array([[30797,    74],
        [21474,  2707]], dtype=int64))

#### 905

In [48]:
mod_905 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID905.hdf5')
mod_905_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id905.hdf5')
mod_905_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id905.hdf5')
mod_905_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id905.hdf5')


In [90]:
evaluate_detection(mod_905, xbasic, ybasic)

(0.127751255502511,
 0.9369474222428824,
 0.9429589206447899,
 0.9373703584679658,
 0.9373703584679658,
 array([[72067,   333],
        [ 8242, 56274]], dtype=int64))

In [91]:
evaluate_detection(mod_905, xplus, yplus)

(0.7682064430751416,
 0.5947077139783268,
 0.779775821415889,
 0.6604301387778827,
 0.6604301387778827,
 array([[30753,   118],
        [18576,  5605]], dtype=int64))

In [92]:
evaluate_detection(mod_905_0, xbasic, ybasic)

(0.05158410316820634,
 0.9718644786208608,
 0.9726456510418081,
 0.9719097841012008,
 0.9719097841012008,
 array([[71882,   518],
        [ 3328, 61188]], dtype=int64))

In [93]:
evaluate_detection(mod_905_0, xplus, yplus)

(0.5988172532153343,
 0.7025726691349677,
 0.8114328738532891,
 0.7333975150766548,
 0.7333975150766548,
 array([[30674,   197],
        [14480,  9701]], dtype=int64))

In [94]:
evaluate_detection(mod_905_1, xbasic, ybasic)

(0.06669663339326679,
 0.9644790216183351,
 0.965863722297186,
 0.9645622133278798,
 0.9645622133278798,
 array([[71851,   549],
        [ 4303, 60213]], dtype=int64))

In [95]:
evaluate_detection(mod_905_1, xplus, yplus)

(0.6100243993217815,
 0.6956234857213938,
 0.8079904892830223,
 0.7281297682191382,
 0.7281297682191382,
 array([[30655,   216],
        [14751,  9430]], dtype=int64))

In [96]:
evaluate_detection(mod_905_2, xbasic, ybasic)

(0.14779279558559116,
 0.9270148210311314,
 0.9349972531459907,
 0.9276271582576178,
 0.9276271582576178,
 array([[72026,   374],
        [ 9535, 54981]], dtype=int64))

In [97]:
evaluate_detection(mod_905_2, xplus, yplus)

(0.7518299491336173,
 0.6058545449783876,
 0.7815541454105245,
 0.6672237157596455,
 0.6672237157596455,
 array([[30731,   140],
        [18180,  6001]], dtype=int64))

#### 906

In [49]:
mod_906 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID906.hdf5')
mod_906_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id906.hdf5')
mod_906_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id906.hdf5')
mod_906_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id906.hdf5')


In [99]:
evaluate_detection(mod_906, xbasic, ybasic)

(0.08554467108934218,
 0.9589758880458612,
 0.9619029276240579,
 0.9591355283531509,
 0.9591355283531509,
 array([[72324,    76],
        [ 5519, 58997]], dtype=int64))

In [100]:
evaluate_detection(mod_906, xplus, yplus)

(0.8397915718952897,
 0.5423410530526002,
 0.7690532664079486,
 0.6298045484269418,
 0.6298045484269418,
 array([[30798,    73],
        [20307,  3874]], dtype=int64))

In [101]:
evaluate_detection(mod_906_0, xbasic, ybasic)

(0.0856996713993428,
 0.958756336989638,
 0.9616707668284087,
 0.9589164159046423,
 0.9589164159046423,
 array([[72304,    96],
        [ 5529, 58987]], dtype=int64))

In [102]:
evaluate_detection(mod_906_0, xplus, yplus)

(0.8210165005582896,
 0.5561779255852202,
 0.7687553137124057,
 0.6373065465378188,
 0.6373065465378188,
 array([[30757,   114],
        [19853,  4328]], dtype=int64))

In [103]:
evaluate_detection(mod_906_1, xbasic, ybasic)

(0.149249798499597,
 0.9278303663821462,
 0.9363902580383465,
 0.928467089310234,
 0.928467089310234,
 array([[72235,   165],
        [ 9629, 54887]], dtype=int64))

In [104]:
evaluate_detection(mod_906_1, xplus, yplus)

(0.9123692154997726,
 0.4833808784232044,
 0.7563384776704783,
 0.5983797137252053,
 0.5983797137252053,
 array([[30823,    48],
        [22062,  2119]], dtype=int64))

In [105]:
evaluate_detection(mod_906_2, xbasic, ybasic)

(0.07584165168330337,
 0.963230834697454,
 0.96548504104383,
 0.9633497911127991,
 0.9633497911127991,
 array([[72275,   125],
        [ 4893, 59623]], dtype=int64))

In [106]:
evaluate_detection(mod_906_2, xplus, yplus)

(0.7423183491170754,
 0.6126983638802298,
 0.7850180013754058,
 0.6717648768437114,
 0.6717648768437114,
 array([[30751,   120],
        [17950,  6231]], dtype=int64))

In [50]:
evaluate_detection(mod_900, xcom, ycom)

(0.2986998493387646,
 0.8423449553013105,
 0.8770446608063805,
 0.8456291108956608,
 0.8456291108956608,
 array([[70965,   720],
        [21412, 50272]], dtype=int64))

In [51]:
evaluate_detection(mod_901, xcom, ycom)

(4.185034317281402e-05,
 0.7588016747310468,
 0.8430914516548553,
 0.7714010699663107,
 0.7714010699663107,
 array([[38914, 32771],
        [    3, 71681]], dtype=int64))

In [52]:
evaluate_detection(mod_902, xcom, ycom)

(0.7866469505049941,
 0.5262685399317407,
 0.7293157797329555,
 0.5954216043914654,
 0.5954216043914654,
 array([[70071,  1614],
        [56390, 15294]], dtype=int64))

In [53]:
evaluate_detection(mod_903, xcom, ycom)

(6.975057195469003e-05,
 0.3334999189634608,
 0.6029523581667466,
 0.5000453375555385,
 0.5000453375555385,
 array([[   12, 71673],
        [    5, 71679]], dtype=int64))

In [54]:
evaluate_detection(mod_904, xcom, ycom)

(0.5454076223425032,
 0.702778012393244,
 0.8166217514654848,
 0.7244174124113302,
 0.7244174124113302,
 array([[71272,   413],
        [39097, 32587]], dtype=int64))

In [55]:
evaluate_detection(mod_905, xcom, ycom)

(0.18232799508955974,
 0.905507496886349,
 0.9194110232244885,
 0.9062489101549149,
 0.9062489101549149,
 array([[71314,   371],
        [13070, 58614]], dtype=int64))

In [56]:
evaluate_detection(mod_906, xcom, ycom)

(0.15991016126332236,
 0.9188588847435221,
 0.9301836404782657,
 0.919368901226904,
 0.919368901226904,
 array([[71588,    97],
        [11463, 60221]], dtype=int64))

#### 1000 (5 nodes)

In [24]:
mod_1000 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID1000.hdf5')
mod_1000_0 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node0id1000.hdf5')
mod_1000_1 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node1id1000.hdf5')
mod_1000_2 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node2id1000.hdf5')
mod_1000_3 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node3id1000.hdf5')
mod_1000_4 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/node4id1000.hdf5')

In [27]:
evaluate_detection(mod_1000, xbasic, ybasic)

(0.9217093434186868,
 0.03429730346229051,
 0.03159272454927669,
 0.03757048117093693,
 0.03757048117093693,
 array([[   93, 72307],
        [59465,  5051]], dtype=int64))

In [33]:
evaluate_detection(mod_1000, xplus, yplus)

(0.18907406641578098,
 0.2336356130445689,
 0.1810240866451773,
 0.3577163409140449,
 0.3577163409140449,
 array([[   84, 30787],
        [ 4572, 19609]], dtype=int64))

In [28]:
evaluate_detection(mod_1000_0, xbasic, ybasic)

(0.9138043276086553,
 0.03777964135857462,
 0.03473752240851977,
 0.04149259399924041,
 0.04149259399924041,
 array([[  120, 72280],
        [58955,  5561]], dtype=int64))

In [34]:
evaluate_detection(mod_1000_0, xplus, yplus)

(0.2131425499359001,
 0.2291125240634923,
 0.17871521092483084,
 0.3474714815083921,
 0.3474714815083921,
 array([[  102, 30769],
        [ 5154, 19027]], dtype=int64))

In [29]:
evaluate_detection(mod_1000_1, xbasic, ybasic)

(0.9404023808047616,
 0.02776373764303839,
 0.026002034700304703,
 0.029901545473136813,
 0.029901545473136813,
 array([[  249, 72151],
        [60671,  3845]], dtype=int64))

In [35]:
evaluate_detection(mod_1000_1, xplus, yplus)

(0.3107811918448369,
 0.20963660332938375,
 0.16709939655739983,
 0.3058562813340115,
 0.3058562813340115,
 array([[  172, 30699],
        [ 7515, 16666]], dtype=int64))

In [30]:
evaluate_detection(mod_1000_2, xbasic, ybasic)

(0.9364498728997458,
 0.029306066699451735,
 0.027371988310696777,
 0.03165444506120541,
 0.03165444506120541,
 array([[  234, 72166],
        [60416,  4100]], dtype=int64))

In [36]:
evaluate_detection(mod_1000_2, xplus, yplus)

(0.29767172573508127,
 0.21159093547586455,
 0.16696024386807953,
 0.3110150403255104,
 0.3110150403255104,
 array([[  139, 30732],
        [ 7198, 16983]], dtype=int64))

In [31]:
evaluate_detection(mod_1000_3, xbasic, ybasic)

(0.9297073594147188,
 0.031337080433207924,
 0.02903038552855499,
 0.03412311198106868,
 0.03412311198106868,
 array([[  137, 72263],
        [59981,  4535]], dtype=int64))

In [37]:
evaluate_detection(mod_1000_3, xplus, yplus)

(0.2705429882965965,
 0.21733646292979078,
 0.17081230114293255,
 0.32271307127806437,
 0.32271307127806437,
 array([[  127, 30744],
        [ 6542, 17639]], dtype=int64))

In [32]:
evaluate_detection(mod_1000_4, xbasic, ybasic)

(0.9350703701407402,
 0.02951085733656612,
 0.027478735812864637,
 0.03196850623740104,
 0.03196850623740104,
 array([[  188, 72212],
        [60327,  4189]], dtype=int64))

In [38]:
evaluate_detection(mod_1000_4, xplus, yplus)

(0.224597824738431,
 0.2265008716626924,
 0.17624056743704708,
 0.3423672164499019,
 0.3423672164499019,
 array([[   98, 30773],
        [ 5431, 18750]], dtype=int64))

In [4]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [5]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcom, ycom = preprocessing(test_complete)

#### 800, 801, 802

In [8]:
mod_800 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID800.hdf5')
mod_801 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID801.hdf5')
mod_802 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/ID802.hdf5')

In [9]:
evaluate_detection(mod_800, xbasic, ybasic)

(0.008232044198895028,
 0.975379422934498,
 0.9758466743376849,
 0.9754082795290543,
 0.9754082795290543,
 array([[61745,  2771],
        [  596, 71804]], dtype=int64))

In [10]:
evaluate_detection(mod_801, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.36580572534487127,
 0.2796203190506846,
 0.5287913757340267,
 0.5287913757340267,
 array([[    0, 64516],
        [    0, 72400]], dtype=int64))

In [11]:
evaluate_detection(mod_802, xbasic, ybasic)

(0.005055248618784531,
 0.9607299641024801,
 0.9628368207667298,
 0.9608519091998013,
 0.9608519091998013,
 array([[59522,  4994],
        [  366, 72034]], dtype=int64))

In [12]:
evaluate_detection(mod_800, xplus, yplus)

(0.010981179747983544,
 0.8056475489028748,
 0.8536176953832524,
 0.8158105064302841,
 0.8158105064302841,
 array([[14380,  9801],
        [  339, 30532]], dtype=int64))

In [13]:
evaluate_detection(mod_801, xplus, yplus)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.4029478640083462,
 0.31445260225958305,
 0.5607607353048027,
 0.5607607353048027,
 array([[    0, 24181],
        [    0, 30871]], dtype=int64))

In [14]:
evaluate_detection(mod_802, xplus, yplus)

(0.010333322535713129,
 0.8239361419268945,
 0.8640875336071753,
 0.8318680520235414,
 0.8318680520235414,
 array([[15244,  8937],
        [  319, 30552]], dtype=int64))

In [15]:
evaluate_detection(mod_800, xcom, ycom)

(0.008788449466415568,
 0.9601272866682927,
 0.9619466801420513,
 0.9601657262030145,
 0.9601657262030145,
 array([[66603,  5081],
        [  630, 71055]], dtype=int64))

In [16]:
evaluate_detection(mod_801, xcom, ycom)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0,
 0.33333720834528774,
 0.25000348751643486,
 0.5000034875042721,
 0.5000034875042721,
 array([[    0, 71684],
        [    0, 71685]], dtype=int64))

In [17]:
evaluate_detection(mod_802, xcom, ycom)

(0.005119620562181767,
 0.9305933206595549,
 0.9380553154521759,
 0.93087766532514,
 0.93087766532514,
 array([[62141,  9543],
        [  367, 71318]], dtype=int64))

### 600 to 606

In [8]:
mod_600 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id600.hdf5')
mod_601 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id601.hdf5')
mod_602 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id602.hdf5')
mod_603 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id603.hdf5')
mod_604 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id604.hdf5')
mod_605 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id605.hdf5')
mod_606 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id606.hdf5')


In [9]:
evaluate_detection(mod_600, xbasic, ybasic)

(0.011325966850828728,
 0.9500586250958318,
 0.9526364067437322,
 0.9502395629437027,
 0.9502395629437027,
 array([[58523,  5993],
        [  820, 71580]], dtype=int64))

In [10]:
evaluate_detection(mod_601, xbasic, ybasic)

(0.9989226519337017,
 0.3030988407521974,
 0.7509555085271363,
 0.4717783166320956,
 0.4717783166320956,
 array([[64516,     0],
        [72322,    78]], dtype=int64))

In [11]:
evaluate_detection(mod_602, xbasic, ybasic)

(0.012361878453038674,
 0.9805860543248166,
 0.9806571853604926,
 0.980593940810424,
 0.980593940810424,
 array([[62754,  1762],
        [  895, 71505]], dtype=int64))

In [12]:
evaluate_detection(mod_603, xbasic, ybasic)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.30184375474744357,
 0.2220375675826312,
 0.4712086242659733,
 0.4712086242659733,
 array([[64516,     0],
        [72400,     0]], dtype=int64))

In [13]:
evaluate_detection(mod_604, xbasic, ybasic)

(0.014074585635359117,
 0.9714436948455951,
 0.9717799989701116,
 0.9714715592041836,
 0.9714715592041836,
 array([[61629,  2887],
        [ 1019, 71381]], dtype=int64))

In [14]:
evaluate_detection(mod_605, xbasic, ybasic)

(0.004019337016574586,
 0.9672131450905261,
 0.9687188040919812,
 0.9672938151859535,
 0.9672938151859535,
 array([[60329,  4187],
        [  291, 72109]], dtype=int64))

In [15]:
evaluate_detection(mod_606, xbasic, ybasic)

(0.011104972375690608,
 0.9817037076793979,
 0.9817804950266569,
 0.9817114142978176,
 0.9817114142978176,
 array([[62816,  1700],
        [  804, 71596]], dtype=int64))

In [16]:
evaluate_detection(mod_600, xplus, yplus)

(0.011791001263321563,
 0.7487386917147426,
 0.8248187420742665,
 0.7682191382692727,
 0.7682191382692727,
 array([[11785, 12396],
        [  364, 30507]], dtype=int64))

In [17]:
evaluate_detection(mod_601, xplus, yplus)

(0.9991253927634349,
 0.2691729649829838,
 0.7537865355700379,
 0.4397297100922764,
 0.4397297100922764,
 array([[24181,     0],
        [30844,    27]], dtype=int64))

In [18]:
evaluate_detection(mod_602, xplus, yplus)

(0.017265394707006575,
 0.9207001222559648,
 0.9266130258528362,
 0.9215650657560125,
 0.9215650657560125,
 array([[20396,  3785],
        [  533, 30338]], dtype=int64))

In [19]:
evaluate_detection(mod_603, xplus, yplus)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.2681015400046588,
 0.19293113164997755,
 0.43923926469519725,
 0.43923926469519725,
 array([[24181,     0],
        [30871,     0]], dtype=int64))

In [20]:
evaluate_detection(mod_604, xplus, yplus)

(0.01946810922872599,
 0.7995370116182215,
 0.8443418509629739,
 0.8096890212889631,
 0.8096890212889631,
 array([[14305,  9876],
        [  601, 30270]], dtype=int64))

In [21]:
evaluate_detection(mod_605, xplus, yplus)

(0.011078358329824107,
 0.7461594868219816,
 0.8243363146066571,
 0.7662573566809562,
 0.7662573566809562,
 array([[11655, 12526],
        [  342, 30529]], dtype=int64))

In [22]:
evaluate_detection(mod_606, xplus, yplus)

(0.00955589388098863,
 0.6504287255082116,
 0.7874536043719509,
 0.6953789144808544,
 0.6953789144808544,
 array([[ 7706, 16475],
        [  295, 30576]], dtype=int64))

In [23]:
evaluate_detection(mod_600, xcom, ycom)

(0.011355234707400432,
 0.9559056848229267,
 0.9579103843600567,
 0.9559528210422058,
 0.9559528210422058,
 array([[66183,  5501],
        [  814, 70871]], dtype=int64))

In [24]:
evaluate_detection(mod_601, xcom, ycom)

(0.9936527864964776,
 0.34034342240552407,
 0.7507959221274985,
 0.5031701413834232,
 0.5031701413834232,
 array([[71684,     0],
        [71230,   455]], dtype=int64))

In [25]:
evaluate_detection(mod_602, xcom, ycom)

(0.013545372114110344,
 0.9704602609035092,
 0.97094925926196,
 0.9704678138230719,
 0.9704678138230719,
 array([[68421,  3263],
        [  971, 70714]], dtype=int64))

In [26]:
evaluate_detection(mod_603, xcom, ycom)

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1.0,
 0.33332945833579397,
 0.24999651250789046,
 0.4999965124957278,
 0.4999965124957278,
 array([[71684,     0],
        [71685,     0]], dtype=int64))

In [27]:
evaluate_detection(mod_604, xcom, ycom)

(0.013698821231777918,
 0.9603692990103223,
 0.9616350614172964,
 0.9603959014849793,
 0.9603959014849793,
 array([[66988,  4696],
        [  982, 70703]], dtype=int64))

In [28]:
evaluate_detection(mod_605, xcom, ycom)

(0.00481272232684662,
 0.9380000619876052,
 0.9439683262907895,
 0.9382014242967448,
 0.9382014242967448,
 array([[63169,  8515],
        [  345, 71340]], dtype=int64))

In [29]:
evaluate_detection(mod_606, xcom, ycom)

(0.011620283183371695,
 0.9641972994200044,
 0.9653047273129904,
 0.9642182061673026,
 0.9642182061673026,
 array([[67387,  4297],
        [  833, 70852]], dtype=int64))

#### Models 700, 701 and 702

In [30]:
mod_700 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id700.hdf5')
mod_701 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id701.hdf5')
mod_702 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/id702.hdf5')


In [31]:
evaluate_detection(mod_700, xcom, ycom)

(0.006263513984794587,
 0.9485102660927153,
 0.9522984846048763,
 0.9486151120535122,
 0.9486151120535122,
 array([[64766,  6918],
        [  449, 71236]], dtype=int64))

In [32]:
evaluate_detection(mod_701, xcom, ycom)

(0.004408174652995745,
 0.9086378332472831,
 0.9218783038238343,
 0.9093179139144445,
 0.9093179139144445,
 array([[58999, 12685],
        [  316, 71369]], dtype=int64))

In [33]:
evaluate_detection(mod_702, xcom, ycom)

(0.018148845644137546,
 0.9879397635678289,
 0.9880125896412548,
 0.9879402102267575,
 0.9879402102267575,
 array([[71256,   428],
        [ 1301, 70384]], dtype=int64))

In [34]:
evaluate_detection(mod_700, xbasic, ybasic )

(0.005455801104972376,
 0.9716245405472854,
 0.9725716710212223,
 0.971676064156125,
 0.971676064156125,
 array([[61033,  3483],
        [  395, 72005]], dtype=int64))

In [35]:
evaluate_detection(mod_700, xplus, yplus )

(0.009847429626510317,
 0.734246008100034,
 0.8203151931332536,
 0.7570115527137978,
 0.7570115527137978,
 array([[11108, 13073],
        [  304, 30567]], dtype=int64))

In [36]:
evaluate_detection(mod_701, xbasic, ybasic )

(0.004102209944751381,
 0.9416874732802485,
 0.9468958940466878,
 0.9420374536211984,
 0.9420374536211984,
 array([[56877,  7639],
        [  297, 72103]], dtype=int64))

In [37]:
evaluate_detection(mod_701, xplus, yplus )

(0.013928930063813936,
 0.8806837182262405,
 0.8976967067608941,
 0.8834374772941945,
 0.8834374772941945,
 array([[18194,  5987],
        [  430, 30441]], dtype=int64))

In [38]:
evaluate_detection(mod_702, xbasic, ybasic )

(0.01819060773480663,
 0.990042161736056,
 0.990217253956467,
 0.9900376873411435,
 0.9900376873411435,
 array([[64469,    47],
        [ 1317, 71083]], dtype=int64))

In [39]:
evaluate_detection(mod_702, xplus, yplus )

(0.018561109131547408,
 0.9468745067838977,
 0.9486164158111176,
 0.9471408849814721,
 0.9471408849814721,
 array([[21844,  2337],
        [  573, 30298]], dtype=int64))

#### TRIM

In [6]:
trim00 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idTRIM00.hdf5')
trim01 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idTRIM01.hdf5')
trim02 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idTRIM02.hdf5')


In [7]:
evaluate_detection(trim00, xbasic, ybasic )

(0.0036187845303867405,
 0.9692174919876617,
 0.9705653078566339,
 0.9692877384673815,
 0.9692877384673815,
 array([[60573,  3943],
        [  262, 72138]], dtype=int64))

In [8]:
evaluate_detection(trim00, xplus, yplus )

(0.005992679213501344,
 0.665614374572885,
 0.7984771010769703,
 0.7067499818353556,
 0.7067499818353556,
 array([[ 8222, 15959],
        [  185, 30686]], dtype=int64))

In [9]:
evaluate_detection(trim01, xbasic, ybasic )

(0.0044060773480662984,
 0.9480098734652984,
 0.9520472639387606,
 0.9482675509071256,
 0.9482675509071256,
 array([[57752,  6764],
        [  319, 72081]], dtype=int64))

In [10]:
evaluate_detection(trim01, xplus, yplus )

(0.01370218003951929,
 0.8722869313316397,
 0.892055665080549,
 0.8755903509409286,
 0.8755903509409286,
 array([[17755,  6426],
        [  423, 30448]], dtype=int64))

In [11]:
evaluate_detection(trim02, xbasic, ybasic )

(0.020124309392265192,
 0.9892761308924688,
 0.9895021323314124,
 0.9892707937713635,
 0.9892707937713635,
 array([[64504,    12],
        [ 1457, 70943]], dtype=int64))

In [12]:
evaluate_detection(trim02, xplus, yplus )

(0.026270609957565354,
 0.9702999397527143,
 0.9702991827410952,
 0.9703008065102086,
 0.9703008065102086,
 array([[23357,   824],
        [  811, 30060]], dtype=int64))

In [13]:
evaluate_detection(trim00, xcom, ycom)

(0.0044500244123596286,
 0.9404631622826496,
 0.9460213145679357,
 0.9406426772872797,
 0.9406426772872797,
 array([[63493,  8191],
        [  319, 71366]], dtype=int64))

In [14]:
evaluate_detection(trim01, xcom, ycom)

(0.004910371765362349,
 0.9154071673070557,
 0.9266286148727427,
 0.9159371970230663,
 0.9159371970230663,
 array([[59984, 11700],
        [  352, 71333]], dtype=int64))

In [15]:
evaluate_detection(trim02, xcom, ycom)

(0.019097440189718907,
 0.9893065571211679,
 0.9894456149808546,
 0.989307311901457,
 0.989307311901457,
 array([[71520,   164],
        [ 1369, 70316]], dtype=int64))

#### MEDIAN

In [50]:
med00 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idMED00.hdf5')
med01 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idMED01.hdf5')
med02 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idMED02.hdf5')


In [51]:
evaluate_detection(med00, xbasic, ybasic )

(0.0032458563535911603,
 0.9685108164478585,
 0.9699706538488233,
 0.968586578632154,
 0.968586578632154,
 array([[60450,  4066],
        [  235, 72165]], dtype=int64))

In [52]:
evaluate_detection(med00, xplus, yplus )

(0.010884001166142982,
 0.7759273251167887,
 0.8385065526992616,
 0.7905798154472136,
 0.7905798154472136,
 array([[12988, 11193],
        [  336, 30535]], dtype=int64))

In [53]:
evaluate_detection(med01, xbasic, ybasic )

(0.004792817679558011,
 0.9522853829097507,
 0.9555799542266943,
 0.9524891174150574,
 0.9524891174150574,
 array([[58358,  6158],
        [  347, 72053]], dtype=int64))

In [54]:
evaluate_detection(med01, xplus, yplus )

(0.015516180233876454,
 0.8645479830216747,
 0.8860697097195477,
 0.8683063285620868,
 0.8683063285620868,
 array([[17410,  6771],
        [  479, 30392]], dtype=int64))

In [55]:
evaluate_detection(med02, xbasic, ybasic )

(0.021104972375690607,
 0.9887945470246919,
 0.9890446953678572,
 0.9887887463846446,
 0.9887887463846446,
 array([[64509,     7],
        [ 1528, 70872]], dtype=int64))

In [56]:
evaluate_detection(med02, xplus, yplus )

(0.02870007450357941,
 0.9838066189632456,
 0.9843397518722772,
 0.9837789726077163,
 0.9837789726077163,
 array([[24174,     7],
        [  886, 29985]], dtype=int64))

In [57]:
evaluate_detection(med00, xcom, ycom)

(0.004310525214480017,
 0.9378625699018214,
 0.9439649392456899,
 0.9380688991344014,
 0.9380688991344014,
 array([[63114,  8570],
        [  309, 71376]], dtype=int64))

In [58]:
evaluate_detection(med01, xcom, ycom)

(0.0051335704819697285,
 0.9212841612252541,
 0.9309317373909491,
 0.9217055290892732,
 0.9217055290892732,
 array([[60827, 10857],
        [  368, 71317]], dtype=int64))

In [59]:
evaluate_detection(med02, xcom, ycom)

(0.01980888609890493,
 0.9900456965845631,
 0.9902371400138832,
 0.9900466628071619,
 0.9900466628071619,
 array([[71677,     7],
        [ 1420, 70265]], dtype=int64))

#### KRUM

In [60]:
krum00 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idKRUM00.hdf5')
krum01 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idKRUM01.hdf5')
krum02 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idKRUM02.hdf5')


In [61]:
evaluate_detection(krum00, xbasic, ybasic )

(0.004378453038674033,
 0.9692349378527148,
 0.9705001595482861,
 0.9693023459639487,
 0.9693023459639487,
 array([[60630,  3886],
        [  317, 72083]], dtype=int64))

In [62]:
evaluate_detection(krum00, xplus, yplus )

(0.0075475365229503415,
 0.7125105552563743,
 0.8138686114556182,
 0.7406452081668241,
 0.7406452081668241,
 array([[10136, 14045],
        [  233, 30638]], dtype=int64))

In [63]:
evaluate_detection(krum01, xbasic, ybasic )

(0.004461325966850829,
 0.955552926164441,
 0.9584258577276722,
 0.9557246779047007,
 0.9557246779047007,
 array([[58777,  5739],
        [  323, 72077]], dtype=int64))

In [64]:
evaluate_detection(krum01, xplus, yplus )

(0.014090894366881539,
 0.9293800076359917,
 0.9344163489049463,
 0.930066119305384,
 0.930066119305384,
 array([[20766,  3415],
        [  435, 30436]], dtype=int64))

In [65]:
evaluate_detection(krum02, xbasic, ybasic )

(0.017803867403314917,
 0.9902391931011721,
 0.9904064503870083,
 0.9902348885448012,
 0.9902348885448012,
 array([[64468,    48],
        [ 1289, 71111]], dtype=int64))

In [66]:
evaluate_detection(krum02, xplus, yplus )

(0.018885037737682613,
 0.9726951004082693,
 0.9727472144071837,
 0.9727167042069317,
 0.9727167042069317,
 array([[23262,   919],
        [  583, 30288]], dtype=int64))

In [67]:
evaluate_detection(krum00, xcom, ycom )

(0.005091720722605845,
 0.9426773587612735,
 0.9476890554678995,
 0.9428328299702167,
 0.9428328299702167,
 array([[63853,  7831],
        [  365, 71320]], dtype=int64))

In [68]:
evaluate_detection(krum01, xcom, ycom )

(0.0053288693590011855,
 0.9264159913216581,
 0.9347770819443417,
 0.9267554352754082,
 0.9267554352754082,
 array([[61565, 10119],
        [  382, 71303]], dtype=int64))

In [69]:
evaluate_detection(krum02, xcom, ycom )

(0.017981446606682012,
 0.9891323871495287,
 0.9892319900707868,
 0.9891329366878474,
 0.9891329366878474,
 array([[71415,   269],
        [ 1289, 70396]], dtype=int64))

#### GEOMETRIC MEDIAN

In [70]:
geo00 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idGEO00.hdf5')
geo01 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idGEO01.hdf5')
geo02 = models.load_model('C:/Users/UX430/Documents/thesis/code/models/idGEO02.hdf5')


In [71]:
evaluate_detection(geo00, xbasic, ybasic )

(0.00712707182320442,
 0.9754574277724276,
 0.9759929096074956,
 0.9754886207601742,
 0.9754886207601742,
 array([[61676,  2840],
        [  516, 71884]], dtype=int64))

In [72]:
evaluate_detection(geo00, xplus, yplus )

(0.008681286644423568,
 0.7656067274071545,
 0.8355189360247318,
 0.7823330669185498,
 0.7823330669185498,
 array([[12466, 11715],
        [  268, 30603]], dtype=int64))

In [73]:
evaluate_detection(geo01, xbasic, ybasic )

(0.004668508287292818,
 0.9440509381249023,
 0.948714588262095,
 0.9443600455753893,
 0.9443600455753893,
 array([[57236,  7280],
        [  338, 72062]], dtype=int64))

In [74]:
evaluate_detection(geo01, xplus, yplus )

(0.014900715882219558,
 0.9165176387721264,
 0.9238694111759653,
 0.9175688440020344,
 0.9175688440020344,
 array([[20103,  4078],
        [  460, 30411]], dtype=int64))

In [75]:
evaluate_detection(geo02, xbasic, ybasic )

(0.01899171270718232,
 0.9898891213014196,
 0.9900909806559807,
 0.9898843086271875,
 0.9898843086271875,
 array([[64506,    10],
        [ 1375, 71025]], dtype=int64))

In [76]:
evaluate_detection(geo02, xplus, yplus )

(0.023549609666029606,
 0.9862322162185762,
 0.9865685957771482,
 0.9862130349487757,
 0.9862130349487757,
 array([[24149,    32],
        [  727, 30144]], dtype=int64))

In [77]:
evaluate_detection(geo00, xcom, ycom )

(0.007574806444862942,
 0.9563495550864797,
 0.9587870328222783,
 0.9564061965975909,
 0.9564061965975909,
 array([[65977,  5707],
        [  543, 71142]], dtype=int64))

In [78]:
evaluate_detection(geo01, xcom, ycom )

(0.005063820883029923,
 0.9129895350603425,
 0.9248167226582382,
 0.9135656941179753,
 0.9135656941179753,
 array([[59655, 12029],
        [  363, 71322]], dtype=int64))

In [79]:
evaluate_detection(geo02, xcom, ycom )

(0.01912534002929483,
 0.990345722631171,
 0.9905226280682651,
 0.9903465881745706,
 0.9903465881745706,
 array([[71671,    13],
        [ 1371, 70314]], dtype=int64))